In [14]:
import os
import argparse
from pydub import AudioSegment
#trim audio and save
import numpy
import math
import scipy.io.wavfile
#model training
import _pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GMM
import warnings#feature utils
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc
def calculate_delta(array):
    """Calculate and returns the delta of given feature vector matrix"""
    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j==rows-1:#pay attenstion here original text i-j rows-1
                second = rows-1
            else:
                second = i+j
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas
 
def extract_features(audio,rate):
    mfcc_feat = mfcc.mfcc(audio,rate, 0.025, 0.01,20,appendEnergy = True)
    mfcc_feat = preprocessing.scale(mfcc_feat)
    #delta = calculate_delta(mfcc_feat)
    #combined = np.hstack((mfcc_feat,delta))
    
    
    return mfcc_feat

9929.18181818182
39


In [40]:
#test models:
import os
import numpy as np
from scipy.io.wavfile import read
import warnings
warnings.filterwarnings("ignore")
import time

#path to training data
def res_feat(modelpath,fpath):
    #get model
    gmm_files = [os.path.join(modelpath,fname) for fname in
                  os.listdir(modelpath) if fname.endswith('.gmm')]
    #Load the Gaussian gender Models
    models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]
    resonances   = [fname.split("\\")[-1].split(".gmm")[0] for fname
                  in gmm_files] 

    # configure for librosa
    y,sr=librosa.load(fpath,sr=44000,res_type="kaiser_fast")
    interval=250/1000*sr
    num=int(len(y)//interval)
    #
    res_features = np.array([])
    for i in range(num+1) :
        start=i*250
        if i==num:
            end=len(y)
        else:
            end=(i+1)*250
        cut=y[start:end]
        vector  = extract_features(cut,sr)
        #section to every 0.25 one rese result out
        #vector=vector.reshape((1, -1))
        log_likelihood = np.zeros(len(models)) 

        for i in range(len(models)):
            gmm    = models[i]  #checking with each model one by one
            scores = np.array(gmm.score(vector))
            log_likelihood[i] = scores.sum()

        if res_features.size == 0:
            res_features = np.array(log_likelihood)
        else:
            res_features = np.vstack((res_features,np.array(log_likelihood)))
    return res_features
            # print(features)

In [41]:
modelpath = "data_resonance\\resonance_models\\"
fpath=r"C:\Users\gaoyu\Desktop\music try\data_resonance\all_types_F_chain.wav"
res_feat(modelpath,fpath).shape

(40, 4)